# Recuperación de la Información y Minería de Texto
# Análisis de noticias

Práctica desarrollada por :

    Cesar González Fernández [cesar.gon.fer@gmail.es]
    Carlos Correa García [carlos.correa.88@gmail.es]

## Índice

1. [Introducción](#introduction)
2. [Casos analizados y ejecutados](#cases)
3. [Conclusiones](#conclusions)

<a id='introduction'></a>
# 1. Introducción

A lo largo de esta práctica, vamos a modelizar y testear diferentes representaciones sobre diferentes noticias recogidas de algunos de los periódicos más influyentes de hoy en día. Esto es, analizaremos diferentes formas de representar un texto como un conjunto de términos que identifiquen a los documentos. De esta forma, al aplicar un algoritmo de clustering sobre esos conjuntos de términos, clasificaremos las noticias mediante la similitud entre términos y obtendremos un resultado de pertenencia a determinados clusters de los diferentes documentos.

Una vez obtenida esa agrupación, mediremos la potencia de la solución mediante la diferencia coseno con el vector de clúster real, donde se representa a qué cluster deben pertenecer las noticias por su temática.

Para ello, utilizaremos dos de las bibliotecas para Python más conocidas de NLP (Natural Language Processing):

* **NLTK**: El kit de herramientas de lenguaje natural, o más comúnmente NLTK, es un conjunto de bibliotecas y programas para el procesamiento del lenguaje natural (PLN) simbólico y estadísticos para el lenguaje de programación Python. NLTK incluye demostraciones gráficas y datos de muestra. Se acompaña de un libro que explica los conceptos subyacentes a las tareas de procesamiento del lenguaje compatibles el toolkit,2​ además de programas de ejemplo.3​ NLTK está destinado a apoyar la investigación y la enseñanza en PLN o áreas muy relacionadas, que incluyen la lingüística empírica, las ciencias cognitivas, la inteligencia artificial, la recuperación de información, y el aprendizaje de la máquina.4​ NLTK se ha utilizado con éxito como herramienta de enseñanza, como una herramienta de estudio individual, y como plataforma para los sistemas de investigación de prototipos y construcción.

* **Spacy**: blablabalbalbaalbalbalbalballbalb


## 1.1 NLTK

blablabalbalbaalbalbalbalballbalb

## 1.2 Spacy

blablabalbalbaalbalbalbalballbalb

<div class  = "alert alert-info"> 
Ejemplo de alert
</div>

Incluimos las bibliotecas y referencias que van a ser necesarias:

In [1]:
from time import time
import os
import pprint
import re

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import *
from sklearn.metrics.cluster import adjusted_rand_score
import nltk
import numpy
import spacy

from utils.parser import html2txt_parser_dir

Para comparar el vector resultado de nuestras pruebas con el resultado real esperado, creamos la variable REFERENCE con la información de clusters. Cada elemento de esta lista corresponde con el identificador de cluster al que pertenece. Si nos paramos a echar un vistazo a las noticias, veremos como efectivamente, la noticia 1, 3, 4, 5 y 20 tratan sobre el mismo tema. De igual manera la noticia 2, 10, 11, 12, 13, 14 y 22, y así sucesivamente.

In [2]:
REFERENCE = [0, 5, 0, 0, 0, 2, 2, 2, 3, 5, 5, 5, 5, 5, 4, 4, 4, 4, 3, 0, 2, 5]

Además, nos hará falta cargar una variable con las utilidades y diccionario de Spacy en el idioma inglés:

In [3]:
nlp = spacy.load('en')

<a id='cases'></a>
# 2. Casos analizados y ejecutados

Hemos analizado diferentes casos, aplicando sobre el texto diferentes transformaciones y analizando resultados, y así ir aplicando sobre el texto las métricas que mejor se comportan en reconocimiento de texto. Comenzamos con el más sencillo, como es simplemente tokenizar, y terminaremos con la obtención de entidades nombradas y nombres más comunes para obtener su frecuencia booleana.

**... pero no adelantemos acontecimientos...!!!**

Vamos a definir los métodos que vamos a utilizar para clusterizar los documentos:

In [6]:
# Function to create a TF vector for one document. For each of
# our unique words, we have a feature which is the tf for that word
# in the current document
def TF(document, unique_terms, collection):
    word_tf = []
    for word in unique_terms:
        word_tf.append(collection.tf(word, document))
    return word_tf

def cluster_texts(texts, cluster_number, distance):
    #Load the list of texts into a TextCollection object.
    collection = nltk.TextCollection(texts)
    print("Creando collecion de %d terminos" % len(collection))

    #get a list of unique terms
    unique_terms = list(set(collection))
    print("Terminos unicos encontrados: ", len(unique_terms))

    ### And here we actually call the function and create our array of vectors.
    vectors = [numpy.array(TF(f,unique_terms, collection)) for f in texts]

    # initialize the clusterer
    clusterer = AgglomerativeClustering(n_clusters=cluster_number,
                                      linkage="average", affinity='cosine')
    clusters = clusterer.fit_predict(vectors)

    return clusters


def timer_decorator(func):
    def func_wrapper(*args):
        start = time()
        res = func(*args)
        end = time()
        print("Tiempo total de ejecución {:.2f}".format((start - end) / 100))
        return res
    return func_wrapper

## 2.1 Tokenización

In [ ]:
#CODIGO

## 2.2 Transformación de tokens a minúsculas

Por lo que extraemos del ejemplo anterior, la tokenización es un método que, sin combinarlo con otros, es bastante malo, puesto que incluso si se obtiene la misma palabra en dos documentos pero difiere una letra minúscula de una mayúscula, el algoritmo de clusterización no va a encontrar similaridad entre ambas palabras, por lo que el ruido entre documentos es mayor.

Es por eso por lo que "normalizamos" todos los tokens a minúscula. De esta forma, obtendremos muy probablemente un número menor de palabras diferentes y por lo tanto, unos clústers un poco mejor definidos.

#CODIGO

Aún habiendo aplicado la transformación a minúsculas, vemos que el Score es muy bajo, por lo que debemos aplicar métricas para obtener similaridades entre documentos algo más complejas.

## 2.3 Eliminación de signos de puntuación (aplicando el punto anterior)

#CODIGO

## 2.4 Eliminación de stopwords en inglés y español (aplicando el punto anterior)

Una vez eliminados los signos de puntuación, que no aportan ningún valor añadido a nuestro análisis de textos, nos puede venir a la mente la idea de eliminar más palabras de nuestros textos que no aportan ningún significado extra al documento y que además, se repiten constantemente en diferentes frases; son las que se denominan "stopwords". Dentro de este conjunto de palabras se encuentran las conjunciones, preposiciones, algunos verbos auxiliares, etc.

Para ello, utilizaremos la lista de palabras que nos proporciona el corpus stopwords de la biblioteca **nltk**:

Estas son las stopwords que eliminaremos del inglés:

In [7]:
print (nltk.corpus.stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Y las del español:

In [8]:
print (nltk.corpus.stopwords.words('spanish'))


['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 

#CODIGO

Tras eliminar todas las stopwords, vemos que el Score obtenido es mayor a ninguna de las anteriores ejecuciones, por lo que mantendremos la tokenización y la eliminación de signos de puntuación y stopwords a lo largo de nuestro análisis de textos.

## 2.5 Traducción con TextBlob (aplicando el punto anterior)

#CODIGO

0.47437425506555425

## 2.6 Traducción con deepl  (aplicando el punto 2.4)

Al igual que en el caso anterior, vamos a utilizar en este caso la bibiolteca **deepl** y analizar si obtenemos unos clusters más definidos en función de los términos obtenidos mediante traducción a inglés con deepl.

#CODIGO

Con la única diferencia de la utilización de una biblioteca u otra de traducción, vemos que se obtiene un Score igual en el caso de utilizar deepl. 

Vamos a utilizar TextBlob para traducir en los siguientes puntos por 2 razones:

    1) En pruebas anteriores, obtuvimos un Score menor con deepl frente al Score de TextBlob. Eso nos hace pensar que la traducción online realizada por deepl ha sido mejorada desde entonces, pero en el mejor de los casos, obtenemos el mismo resultado que con TextBlob, por lo que descartaremos deepl.
    
    2) La traducción con TextBlob es mucho más liviana y su tiempo de ejecución es considerablemente menor.




## 2.7 Eliminación de stopwords ampliadas (aplicando el punto anterior (deepl))

#CODIGO

## 2.8 Utilización de entidades nombradas

Es natural pensar que existan ciertas palabras singulares en el texto con una importancia muy alta. Por ello, la identificación de entidades nombradas es un método común en procesamiento de texto. Ese método reconoce personas relevantes en el texto, organizaciones, lugares, expresiones de tiempo, cantidades, etc.

Por ello, vamos a hacer uso de la biblioteca spacy y su reconocimiento de entidades nombradas, mediante el atributo "ents".

#CODIGO

En este caso, el Score sube hasta 0.815913688469319, reduciendo a 4 las noticias erróneamente clasificadas.

## 2.9 Utilización de entidades nombradas preseleccionadas

#CODIGO

## 2.10 Utilización de entidades nombradas preseleccionadas y 5 tokens más comunes por texto

Si analizamos cualquier texto o noticia, podemos ver que además de ciertas entidades nombradas interesantes, hay otras palabras comunes que son de mucho interés y aportan gran cantidad de información. Por ejemplo, en noticias sobre el escándalo de Oxfam, podría interesarnos sustantivos (nombres comunes) como "abuso", "sexo" o "verguenza". Esas palabras tienen un peso muy alto en la noticia y que conjuntamente con las entidades nombradas, nos pueden ofrecer un conjunto de términos mucho más completo.

Por ello, vamos a contar el número de sustantivos que aparecen en el texto y a quedarnos con los 5 más significativos.

#CODIGO

Observamos en los resultados de la ejecución que el Score sigue siendo el mismo y sigue existiendo el error en la clusterización que aún teníamos en el caso anterior. Parece que el número de entidades nombradas es muy superior a los 5 sustantivos más comunes de texto.

## 2.11 Utilización de entidades nombradas preseleccionadas y 5 tokens más comunes por texto mediante frecuencia booleana

Puesto que el caso anterior, aunque el Score sea el mismo y el error en la clusterización siga vigente, la información que se aporta de cada noticia es claramente mayor que sin tener en cuenta los nombres más repetidos en el texto. El problema es que las entidades nombradas pueden estar siendo repetidas mientras que los 5 tokens NOUN únicamente aparecen 1 vez cada uno, por lo que se le restan apariciones en el texto y se está dando automáticamente más peso a las entidades nombradas.

Por ello, vamos a ejecutar el mismo algoritmo que el caso anterior con la única diferencia que se va a devolver una lista de términos sin repetición.

#CODIGO

<a id='conclusions'></a>
# 3 Conclusiones

CONCLUSIONES

#CODIGO

#METER GRAFICA CON TODAS LAS EJECUCIONES

In [ ]:
#PAJOTE FINAL